In [16]:
# This version uses abstracted functions that actually work to load X and Y training data into arrays of the right shape
# data successfully flows through LSTM connected to a flatten and dense layer
# this version is trained on all of part 1 data and the model is saved in a local file

# Here I will be building out the architecture of the first classification LSTM
# At each time step, this LSTM will take in a vector representing the extracted audio and visual features from Liris-ACCEDE
# Its goal is to output whether or not the movie induces fear at each time step


# First, import necessary libraries
import tensorflow as tf
import numpy as np
import keras.backend as K

In [2]:
# setting up the keras stuff
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import LSTM
# my custom data_utils file
from data_utils_local08 import *

Using TensorFlow backend.


In [3]:
# uploading the X values
X_input = load_Xinput(get_fc6_directory(7))
print(X_input.shape)

(212, 4096)


In [4]:
# uploading the Y values
y_data_input = fear_oneHot(212, 'fear_annotations_part01/MEDIAEVAL18_7_Fear.txt')
print(y_data_input.shape)

(212,)


In [5]:
# Uploading part01 of the local fc6 data

timesteps = 212   # the number of seconds in movie 07 --> i will figure out how to programmatically get this value
data_dim = 4096    # the number of output values from VGG16 layer fc6 --> switch to programmatic later
num_movies = 4
batch_size = 7
num_epochs = 5
validation_num = 3

# set up the X_train_data master array
X_train_data = np.zeros([num_movies, timesteps, data_dim]) #oooooh this array will have to be as long as the longest
                                                            # movie and padded with zeros --> this won't cause problems
                                                            # right?
X_valid_data = np.zeros([validation_num, timesteps, data_dim])
        
# for each movie number between and including 7 and 13
for num in range(num_movies):
    # load the X_input data
    X_input = load_Xinput(get_fc6_directory(7 + num))
    # put this X_input data into the X_train_data array
    X_train_data[num, :, :] = X_input
print(X_train_data.shape)



(4, 212, 4096)


In [6]:
# loading X validation set
X_valid_data = np.zeros([validation_num, timesteps, data_dim])

for num in range(validation_num):
    # load the X_input data
    X_valid = load_Xinput(get_fc6_directory(7 + num_movies + num))
    # put this X_input data into the X_train_data array
    X_valid_data[num, :, :] = X_valid
print(X_valid_data.shape)


(3, 212, 4096)


In [7]:
!pwd

/Users/chloeloughridge/AIGrant_LSTMdev


In [8]:
# uploading the y data

# set up y_train_data master array
Y_train_data = np.zeros([num_movies, timesteps])

# for each movie number between and including 7 and 13
for num in range(num_movies):
    # create the appropriate path to the fear annotation data
    #print(num)
    path = os.path.join('fear_annotations_part01/MEDIAEVAL18_{}_Fear.txt'.format(7+num))
    # create a one-hot vector
    y_data = fear_oneHot(timesteps, path)
    # add this one-hot vector to y_train_data
    Y_train_data[num, :] = y_data
print(Y_train_data[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [9]:
# upload the Y validation set
Y_valid_data = np.zeros([validation_num, timesteps])

# for each movie number in validation set
for num in range(validation_num):
    # create the appropriate path to the fear annotation data
    #print(num)
    path = os.path.join('fear_annotations_part01/MEDIAEVAL18_{}_Fear.txt'.format(7+ num_movies + num))
    # create a one-hot vector
    y_valid = fear_oneHot(timesteps, path)
    # add this one-hot vector to y_train_data
    Y_valid_data[num, :] = y_valid
print(Y_valid_data[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [10]:
# constructing a many-to-one LSTM model in keras --> inspiration: https://stackoverflow.com/questions/43034960/many-to-one-and-many-to-many-lstm-examples-in-keras
# i will start by training a model on only the VGG16 fc6 layer output (that's just one feature)
# should I eventually abstract this LSTM model? Create its own object file?
model = Sequential()
model.add(LSTM(212, return_sequences=True,
               input_shape=(timesteps, data_dim)))
# going to try adding a flatten layer in here
model.add(Flatten()) # I got this from a github thing, but I still don't completely understand why it works
# add the final dense layer and then softmax
model.add(Dense(212, activation='sigmoid'))
# going to add a softmax activation to this
#model.add(Activation('softmax'))

In [17]:
# a new metric of evaluation! the F-score!
def FScore2(y_true, y_pred):
    '''
    The F score, beta=2
    '''
    B2 = K.variable(4)
    OnePlusB2 = K.variable(5)
    pred = K.round(y_pred)
    tp = K.sum(K.cast(K.less(K.abs(pred - K.clip(y_true, .5, 1.)), 0.01), 'float32'), -1)
    fp = K.sum(K.cast(K.greater(pred - y_true, 0.1), 'float32'), -1)
    fn = K.sum(K.cast(K.less(pred - y_true, -0.1), 'float32'), -1)

    f2 = OnePlusB2 * tp / (OnePlusB2 * tp + B2 * fn + fp)

    return K.mean(f2)

In [20]:
# compiling LSTM model
# note that Ng used an Adam optimizer and categorical cross-entropy loss
# but this is a binary classification problem so I think the parameters below should suffice
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['binary_accuracy', FScore2])

In [21]:
# running the LSTM model
model.fit(X_train_data, Y_train_data, epochs = 20, validation_data=(X_valid_data, Y_valid_data))
print("finished training!")

Train on 4 samples, validate on 3 samples
Epoch 1/20
4/4 [==============================] - 3s 862ms/step - loss: 4.0847e-06 - binary_accuracy: 1.0000 - FScore2: 1.0000 - val_loss: 2.3184 - val_binary_accuracy: 0.7091 - val_FScore2: 0.1421
Epoch 2/20
4/4 [==============================] - 2s 601ms/step - loss: 1.5648e-06 - binary_accuracy: 1.0000 - FScore2: 1.0000 - val_loss: 2.3866 - val_binary_accuracy: 0.7013 - val_FScore2: 0.1224
Epoch 3/20
4/4 [==============================] - 2s 612ms/step - loss: 4.2172e-07 - binary_accuracy: 1.0000 - FScore2: 1.0000 - val_loss: 2.4186 - val_binary_accuracy: 0.7060 - val_FScore2: 0.1201
Epoch 4/20
4/4 [==============================] - 2s 601ms/step - loss: 3.0463e-07 - binary_accuracy: 1.0000 - FScore2: 1.0000 - val_loss: 2.4412 - val_binary_accuracy: 0.7044 - val_FScore2: 0.1235
Epoch 5/20
4/4 [==============================] - 2s 609ms/step - loss: 2.4863e-07 - binary_accuracy: 1.0000 - FScore2: 1.0000 - val_loss: 2.4482 - val_binary_accurac

In [15]:
# trying to view the model output
out = model.predict(X_train_data)
print("model prediction:")
print(out[0])
print("target:")
print(Y_train_data[0])

print("before 64:")
print(out[0][63])
print("64:")
print(out[0][65])

print("rounded")
print(np.round(out)[0])

model prediction:
[2.25139465e-05 9.77822765e-07 1.55764565e-06 9.70406109e-06
 6.14908140e-06 2.91141873e-06 5.22274013e-06 1.31821944e-05
 3.59225123e-06 1.60548734e-05 1.79300896e-05 1.42105273e-05
 6.05305013e-06 1.40088814e-05 2.13852745e-05 3.10756695e-06
 3.44362502e-06 1.65802576e-05 1.54017562e-05 3.70630232e-06
 5.05536718e-06 1.67455983e-05 6.35322112e-06 1.66260652e-05
 1.50669193e-05 3.79963308e-06 2.20809379e-05 1.12312728e-05
 1.93098022e-05 1.42896597e-05 3.82243934e-06 2.33241171e-05
 4.73718392e-04 2.56292173e-04 1.77335933e-05 2.45983887e-04
 8.03970124e-06 1.66992960e-03 1.09345077e-04 3.60266713e-04
 2.22114377e-05 5.07977093e-04 4.92297841e-06 4.86800127e-04
 5.38802260e-06 8.06433491e-06 1.23085229e-05 6.87312968e-06
 1.83838012e-04 2.77133251e-04 7.47776676e-06 1.53146088e-04
 9.04848450e-04 2.22479124e-04 1.79624883e-04 4.24964412e-04
 1.01324520e-03 1.35171483e-03 1.84225471e-04 1.22018108e-04
 2.12393847e-04 4.49114188e-04 2.76052597e-04 6.12464100e-06
 9.990

In [14]:
#try visualizing this model at some point?